In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

import pandas as pd
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

nlp = spacy.load('en')

dtypes ={
    'id': np.uint32,
    'qid1': np.uint32,
    'qid2': np.uint32,
    'question1': np.str,
    'question2': np.str,
    'is_duplicate': np.uint8
}

df = pd.read_csv('../../dataset/quora_train.csv.zip', dtype=dtypes, compression='zip', usecols=['question1', 'question2', 'is_duplicate'], nrows=10000)

In [2]:
def pad(v):
    zeros = np.zeros([100, 300])
    zeros[:v.shape[0],:v.shape[1]] = v
    return zeros
    
q1s = df['question1'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)
q2s = df['question1'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)

q1m = np.concatenate(q1s.values).flatten().reshape([-1, 100, 300])
q2m = np.concatenate(q2s.values).flatten().reshape([-1, 100, 300])

labels = df['is_duplicate'].values

KeyboardInterrupt: 

In [ ]:
train_q1m, test_q1m = q1m[:7000], q1m[7000:]
train_q2m, test_q2m = q2m[:7000], q2m[7000:]
train_labels, test_labels = labels[:7000], labels[7000:]

In [ ]:
tf.reset_default_graph()

Q1 = tf.placeholder(tf.float32, shape=[None, 100, 300])
Q2 = tf.placeholder(tf.float32, shape=[None, 100, 300])
is_dup = tf.placeholder(tf.uint8, shape=[None])
dup_oh = tf.one_hot(is_dup, 2)

cell1 = rnn.BasicLSTMCell(128, state_is_tuple=True)
cell2 = rnn.BasicLSTMCell(128, state_is_tuple=True, reuse=True)

state = cell1.zero_state(100, tf.float32)

#Q1 = tf.transpose(Q1, [1, 0, 2])
#Q2 = tf.transpose(Q2, [1, 0, 2])

outputs1, state = tf.nn.dynamic_rnn(cell1, Q1, initial_state=state, time_major=True)
outputs2, state = tf.nn.dynamic_rnn(cell2, Q2, initial_state=state, time_major=True)

output1 = outputs1[:,-1,:]
output2 = outputs2[:,-1,:]

out = tf.concat([output1, output2], axis=1)

In [ ]:
W1 = tf.get_variable(shape=[256, 256], initializer=tf.contrib.layers.xavier_initializer(), name='weight1')
b1 = tf.get_variable(shape=[256], initializer=tf.contrib.layers.xavier_initializer(), name='bias1')
y1 = tf.nn.relu(tf.matmul(out, W1) + b1)

W2 = tf.get_variable(shape=[256, 2], initializer=tf.contrib.layers.xavier_initializer(), name='weight2')
b2 = tf.get_variable(shape=[2], initializer=tf.contrib.layers.xavier_initializer(), name='bias2')
y2 = tf.matmul(y1, W2) + b2

In [ ]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=y2, labels=dup_oh)
cost = tf.reduce_mean(cost)

optimizer = tf.train.AdamOptimizer(learning_rate=0.003).minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
from tqdm import tqdm

train_dict = {Q1: train_q1m, Q2: train_q2m, is_dup: train_labels}
test_dict = {Q1: test_q1m, Q2: test_q2m, is_dup: test_labels}

for i in tqdm(range(10000)):
    sess.run(optimizer, feed_dict=train_dict)
    if i % 10 == 0:
        print(sess.run(cost, feed_dict=train_dict), sess.run(cost, feed_dict=test_dict))